In [ ]:
import math

In [ ]:
def _solar_declination(day_of_year):
   
    if not 1 <= day_of_year <= 366:
        raise ValueError("Day of the year must be in the range [1-366]: "
                         "{0!r}".format(day_of_year))

    return 0.409 * np.sin(((2.0 * np.pi / 365.0) * day_of_year - 1.39))

In [ ]:
def _sunset_hour_angle(latitude_radians, solar_declination_radians):
    
    cos_sunset_hour_angle = -np.tan(latitude_radians) * np.tan(solar_declination_radians)

    # If the sunset hour angle is >= 1 there is no sunset, i.e. 24 hours of daylight
    # If the sunset hour angle is <= 1 there is no sunrise, i.e. 24 hours of darkness
    # See http://www.itacanet.org/the-sun-as-a-source-of-energy/part-3-calculating-solar-angles/
    # Domain of acos is -1 <= x <= 1 radians (this is not mentioned in FAO-56!)
    return math.acos(min(max(cos_sunset_hour_angle, -1.0), 1.0))

In [ ]:
def _daylight_hours(sunset_hour_angle_radians):

   # validate the sunset hour angle argument, which has a valid
    # range of 0 to pi radians (180 degrees), inclusive
    # see http://mypages.iit.edu/~maslanka/SolarGeo.pdf
    if not 0.0 <= sunset_hour_angle_radians <= math.pi:
        raise ValueError(f"sunset hour angle outside valid range [0.0 to {math.pi!r}]"
                         f": {sunset_hour_angle_radians!r}")

    # calculate daylight hours from the sunset hour angle
    return (24.0 / math.pi) * sunset_hour_angle_radians

In [ ]:
def _monthly_mean_daylight_hours(latitude_radians: float, leap=False, week=False):
    """
    :param latitude_radians: latitude in radians
    :param leap: whether or not values should be computed specific to leap years
    :return: the mean daily daylight hours for each calendar month of a year
    :rtype: numpy.ndarray of floats, 1-D with shape: (12,)
    """
    
    _MONTH_DAYS_NONLEAP = np.array([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])
    _MONTH_DAYS_LEAP = np.array([31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])
    
    _WEEK_DAYS_NONLEAP = np.array([8, 7, 7, 9, 
                                   8, 7, 7, 6, 
                                   8, 7, 7, 9, 
                                   8, 7, 7, 8, 
                                   8, 7, 7, 9, 
                                   8, 7, 7, 8, 
                                   8, 7, 7, 9,  
                                   8, 7, 7, 9, 
                                   8, 7, 7, 8, 
                                   8, 7, 7, 9, 
                                   8, 7, 7, 8, 
                                   8, 7, 7, 9])
    _WEEK_DAYS_LEAP = np.array([8, 7, 7, 9, 
                                8, 7, 7, 7, 
                                8, 7, 7, 9, 
                                8, 7, 7, 8, 
                                8, 7, 7, 9, 
                                8, 7, 7, 8, 
                                8, 7, 7, 9,  
                                8, 7, 7, 9, 
                                8, 7, 7, 8, 
                                8, 7, 7, 9, 
                                8, 7, 7, 8, 
                                8, 7, 7, 9])
    
    
    # get the array of days for each month based
    # on whether or not we're in a leap year
    if not leap:
        month_days = _MONTH_DAYS_NONLEAP
        week_days  = _WEEK_DAYS_NONLEAP
    else:
        month_days = _MONTH_DAYS_LEAP
        week_days  = _WEEK_DAYS_LEAP

    # allocate an array of daylight hours for each of the 12 months of the year
  

    # loop over each calendar month to calculate the daylight hours for the month
    if week:
        monthly_mean_dlh = np.zeros((12*4,))
        day_of_year = 1
        for i, days_in_week in enumerate(week_days):
            cumulative_daylight_hours = 0.0  # cumulative daylight hours for the month
            for _ in range(1, days_in_week + 1):
                daily_solar_declination = _solar_declination(day_of_year)
                daily_sunset_hour_angle = \
                    _sunset_hour_angle(latitude_radians, daily_solar_declination)
                cumulative_daylight_hours += _daylight_hours(daily_sunset_hour_angle)
                day_of_year += 1
            monthly_mean_dlh[i] = cumulative_daylight_hours / days_in_week
        
    else:
        monthly_mean_dlh = np.zeros((12,))
        day_of_year = 1
        for i, days_in_month in enumerate(month_days):
            cumulative_daylight_hours = 0.0  # cumulative daylight hours for the month
            for _ in range(1, days_in_month + 1):
                daily_solar_declination = _solar_declination(day_of_year)
                daily_sunset_hour_angle = \
                    _sunset_hour_angle(latitude_radians, daily_solar_declination)
                cumulative_daylight_hours += _daylight_hours(daily_sunset_hour_angle)
                day_of_year += 1
            monthly_mean_dlh[i] = cumulative_daylight_hours / days_in_month
       
    return monthly_mean_dlh

In [ ]:
def et_calculation(temps_celsius: np.ndarray,
                   latitude_degrees: float,
                   data_start_year: int,
                   shape: int,
                   number_of_years: int):
    
    import calendar
    
    original_length = temps_celsius.size

    # validate the input data array
    temps_celsius = np.reshape(temps_celsius, (number_of_years, shape))

    # convert the latitude from degrees to radians
    latitude_radians = np.radians(latitude_degrees)
    
    temps_celsius[temps_celsius < 0] = 0.0
    
    mean_temps = np.nanmean(temps_celsius, axis=0)
    
    # calculate the heat index (I)
    heat_index = np.sum(np.power(mean_temps / 5.0, 1.514))
    
    # calculate the a coefficient
    a = ((6.75e-07 * heat_index ** 3)
         - (7.71e-05 * heat_index ** 2)
         + (1.792e-02 * heat_index)
         + 0.49239)

    # get mean daylight hours for both normal and leap years
    if shape != 12:
        mean_daylight_hours_nonleap = \
            np.array(_monthly_mean_daylight_hours(latitude_radians, False, True))
        mean_daylight_hours_leap = \
            np.array(_monthly_mean_daylight_hours(latitude_radians, True, True))
    else:
        mean_daylight_hours_nonleap = \
            np.array(_monthly_mean_daylight_hours(latitude_radians, False, False))
        mean_daylight_hours_leap = \
            np.array(_monthly_mean_daylight_hours(latitude_radians, True, False))
    
    # allocate the PET array we'll fill
    pet = np.full(temps_celsius.shape, np.NaN)
   
    _MONTH_DAYS_NONLEAP = np.array([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])
    _MONTH_DAYS_LEAP = np.array([31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])
    _WEEK_DAYS_NONLEAP = np.array([8, 7, 7, 9, 
                                   8, 7, 7, 6, 
                                   8, 7, 7, 9, 
                                   8, 7, 7, 8, 
                                   8, 7, 7, 9, 
                                   8, 7, 7, 8, 
                                   8, 7, 7, 9,  
                                   8, 7, 7, 9, 
                                   8, 7, 7, 8, 
                                   8, 7, 7, 9, 
                                   8, 7, 7, 8, 
                                   8, 7, 7, 9])
    _WEEK_DAYS_LEAP = np.array([8, 7, 7, 9, 
                                8, 7, 7, 7, 
                                8, 7, 7, 9, 
                                8, 7, 7, 8, 
                                8, 7, 7, 9, 
                                8, 7, 7, 8, 
                                8, 7, 7, 9,  
                                8, 7, 7, 9, 
                                8, 7, 7, 8, 
                                8, 7, 7, 9, 
                                8, 7, 7, 8, 
                                8, 7, 7, 9])
    
    for year in range(temps_celsius.shape[0]):

        if eto.calendar.isleap(data_start_year + year):
            month_days          = _MONTH_DAYS_LEAP
            week_days           = _WEEK_DAYS_LEAP
            mean_daylight_hours = mean_daylight_hours_leap
        else:
            month_days          = _MONTH_DAYS_NONLEAP
            week_days           = _WEEK_DAYS_NONLEAP
            mean_daylight_hours = mean_daylight_hours_nonleap

        # calculate the Thornthwaite equation
        
        if shape != 12:
            pet[year, :] = (
                16
                * (mean_daylight_hours / 12.0)
                * (week_days / 7.0)
                * ((10.0 * temps_celsius[year, :] / heat_index) ** a))
        else:
            pet[year, :] = (
                16
                * (mean_daylight_hours / 12.0)
                * (month_days / 30.0)
                * ((10.0 * temps_celsius[year, :] / heat_index) ** a)
                
        )
        
    # reshape the dataset from (years, 12) into (months),
    # i.e. convert from 2-D to 1-D, and truncate to the original length
    return pet.reshape(-1)[0:original_length]